In [1]:
%%capture
import os
from IPython.display import Javascript

os.environ['R_HOME'] = 'C:\Program Files\R\R-4.4.2'

In [2]:
# !python -m ipykernel install --user --name=ericapp --display-name "Python3(ericapp)"

In [3]:
%load_ext rpy2.ipython
%run -i ./MDT_Functions_copy.py

C:\Users\alifr\anaconda3\envs\ericapp\Lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [4]:
0.01### INPUT DATA
inputSpatial()

# SIMULATE RESPONSE
simulateReponse()

# ## simulate resposne
# Javascript("Jupyter.notebook.execute_cells([2])")

D:\ABMI_Official\EricCodePractice\EWNeilsen_OccupancySpatialPowerApp
The study area has (417996.4415538451, 795945.195356781, 1126884.4543278955, 1581144.825542611). 
It has 76 rows and 91 columns.

------------------
Finished inputing Use vars: ['elevation', 'water_proximity']



C:\Users\alifr\anaconda3\envs\ericapp\Lib\site-packages\osgeo\gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [5]:
# print(saAreaKM)

In [6]:
## SIMULATE OCCUANPCY DATA
simulateOccupancyData()

D:\ABMI_Official\EricCodePractice\EWNeilsen_OccupancySpatialPowerApp


In [7]:
%R -i cwd
%R -i trueOcc
%R -i popPX
%R -i N
%R -i meanDetection

In [11]:
%%R

print(cwd)
print(trueOcc)
print(popPX)
print(N)
print(meanDetection)

[1] "D:\\ABMI_Official\\EricCodePractice\\EWNeilsen_OccupancySpatialPowerApp"
[1] 0.51
[1] 1
[1] 44125.57
[1] 0.5400442


In [13]:
%%R

library(unmarked)
options(warn=2)

getParams = function(modObject){
    
    outParams = list()
    psiTab = predict(modObject, type="state",newdata=data.frame(1))
    outParams$psi = psiTab$Predicted
    outParams$psiSE = psiTab$SE
    outParams$psiBias = outParams$psi - trueOcc
    
    pTab = predict(modObject, type="det",newdata=data.frame(1))
    outParams$p = pTab$Predicted
    outParams$pSE = pTab$SE
    outParams$pBias = outParams$p - meanDetection
    
    return(outParams)
    }


OccOutTab = data.frame(CamN = NA, IntervalsN = NA, Response = NA, Estimate = NA,
                       SE = NA, Bias = NA, inSig = NA)


dhScenDirs = list.dirs(paste0(cwd,'/Data/DetectionHistories'),recursive = FALSE)  
# print(dhScenDirs)
scenCount = 0
FailCount = 0 

for (dir in dhScenDirs){    

    scenI = unlist(strsplit(dir,"/"))
    scen = scenI[length(scenI)]
    scen = unlist(strsplit(scen,"_"))
    CameraNumber = scen[1]
    VisitsNumber = scen[2]

    dhScens = list.files(dir,pattern="\\.csv$")
    
    for (scen in dhScens){        
        dh = read.csv(paste0(dir,"/",scen),header=T)

        ## fit model to scenario data
        umf <- unmarkedFrameOccu(y=as.matrix(dh)) # organize data
        fm <- try(occu(~1 ~1, umf)) # fit a model
        modOut = try(coef(fm)) 
#         print('')
#         print('')
#         print(class(fm))
        
        if(class(modOut) != "try-error" ){    

#             ## model outputs
#             psiTab = predict(fm, type="state",newdata=data.frame(1))
#             psi = psiTab$Predicted
#             psiSE = psiTab$SE
#             psiBias = psi - trueOcc
#             pTab = predict(fm, type="det",newdata=data.frame(1))
#             p = pTab$Predicted
#             pSE = pTab$SE
#             pBias = p - meanDetection
            
            OP = try(getParams(fm))
            if(class(OP)!= "try-error"){

                ## occupancy info
                scenCount = scenCount + 1
                OccOutTab[scenCount,"CamN"] = CameraNumber
                OccOutTab[scenCount,"IntervalsN"] = VisitsNumber
                OccOutTab[scenCount,"Response"] = "psi"
                OccOutTab[scenCount,"Estimate"] = OP$psi
                OccOutTab[scenCount,"SE"] = OP$psiSE
                OccOutTab[scenCount,"Bias"] = OP$psiBias
                OccOutTab[scenCount,"inSig"] = dplyr::between(0,OP$psi-(OP$psiSE*2.58),OP$psi+(OP$psiSE*2.58))

                ## detection info
                scenCount = scenCount + 1
                OccOutTab[scenCount,"CamN"] = CameraNumber
                OccOutTab[scenCount,"IntervalsN"] = VisitsNumber
                OccOutTab[scenCount,"Response"] = "p"
                OccOutTab[scenCount,"Estimate"] = OP$p
                OccOutTab[scenCount,"SE"] = OP$pSE
                OccOutTab[scenCount,"Bias"] = OP$pBias
                OccOutTab[scenCount,"inSig"] = dplyr::between(0,OP$p-(OP$pSE*2.58),OP$p+(OP$pSE*2.58))
            }
            else{
                FailCount = FailCount + 1 
                print("Model failed")
            }
        }
        else{
            FailCount = FailCount + 1 
            print("Model failed")
        }
    }
}



print("Fit occupancy models to all detection histories.")
print(paste0(FailCount," models failed."))

 [1] "D:\\ABMI_Official\\EricCodePractice\\EWNeilsen_OccupancySpatialPowerApp/Data/DetectionHistories/12_2"
 [2] "D:\\ABMI_Official\\EricCodePractice\\EWNeilsen_OccupancySpatialPowerApp/Data/DetectionHistories/12_3"
 [3] "D:\\ABMI_Official\\EricCodePractice\\EWNeilsen_OccupancySpatialPowerApp/Data/DetectionHistories/12_4"
 [4] "D:\\ABMI_Official\\EricCodePractice\\EWNeilsen_OccupancySpatialPowerApp/Data/DetectionHistories/12_5"
 [5] "D:\\ABMI_Official\\EricCodePractice\\EWNeilsen_OccupancySpatialPowerApp/Data/DetectionHistories/12_6"
 [6] "D:\\ABMI_Official\\EricCodePractice\\EWNeilsen_OccupancySpatialPowerApp/Data/DetectionHistories/21_4"
 [7] "D:\\ABMI_Official\\EricCodePractice\\EWNeilsen_OccupancySpatialPowerApp/Data/DetectionHistories/21_6"
 [8] "D:\\ABMI_Official\\EricCodePractice\\EWNeilsen_OccupancySpatialPowerApp/Data/DetectionHistories/23_2"
 [9] "D:\\ABMI_Official\\EricCodePractice\\EWNeilsen_OccupancySpatialPowerApp/Data/DetectionHistories/23_3"
[10] "D:\\ABMI_Official\\Eri

In [7]:
# !pip install ipyleaflet

In [28]:
plot_output = widgets.Output()

In [29]:
def update_center(value):
    with plot_output:
        plot_output.clear_output(wait=True)
        print(value)

In [30]:
def on_location_changed(event):
    value = marker.location
    update_center(value)

marker.observe(on_location_changed, 'location')

In [31]:
import ipywidgets as widgets
from ipyleaflet import Map, Marker
from IPython.display import display

center = (52.204793, 360.121558)
map = Map(center=center, zoom=12)

# Add a draggable marker to the map
# Dragging the marker updates the marker.location value in Python
marker = Marker(location=center, draggable=True)
# print(marker.location)
map.add_control(marker)

display(map)

Map(center=[52.204793, 360.121558], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…